In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dropout
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

# Verileri yükle
data = pd.read_excel("train.xlsx")
data = data.drop(["id", "temp", "w_status"], axis=1)

# 'date' sütununu datetime formatına çevirme
data['date_v'] = pd.to_datetime(data['date_v'])

# 'date' sütununu Unix zaman damgasına çevirme
data['unix_time'] = data['date_v'].apply(lambda x: int((x - datetime(1970, 1, 1)).total_seconds()))

# Gereksiz sütunu ve orijinal tarih sütununu kaldırma
data = data.drop(["date_v"], axis=1)

# MinMaxScaler ile 'yukh', 'reg' ve 'power' sütunlarını ölçeklendirme
scaler_yukh = MinMaxScaler()
scaler_reg = MinMaxScaler()
scaler_power = MinMaxScaler()

data['yukh_scaled'] = scaler_yukh.fit_transform(data[['yukh']])
data['reg_scaled'] = scaler_reg.fit_transform(data[['reg']])
data['power_scaled'] = scaler_power.fit_transform(data[['power']])

# Ölçeklenmiş sütunları veriden kaldırma
data = data.drop(["yukh", "reg", "power"], axis=1)

# Zaman serisi formatına dönüştürme
def create_time_series_data(data, window_size):
    X, y = [], []

    for i in range(len(data) - window_size):
        # Pencere boyutu kadar veriyi seçme
        window_yukh = data['yukh_scaled'][i:i+window_size].values
        window_reg = data['reg_scaled'][i:i+window_size].values
        window_power = data['power_scaled'][i:i+window_size].values
        target = data['yukh_scaled'][i + window_size]

        X.append(np.column_stack((window_yukh, window_reg, window_power)))
        y.append(target)

    return np.array(X), np.array(y)

# Pencere boyutunu belirleme
window_size = 64

# Giriş ve hedef verilerini oluşturma
X, y = create_time_series_data(data, window_size)

# Veriyi eğitim ve test setlerine ayırma
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

# Modeli oluşturma
model = Sequential()
model.add(LSTM(units=50, activation='relu', return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(units=50, activation='relu', return_sequences=True))
model.add(LSTM(units=50, activation='relu'))
model.add(Dropout(0.2))  # Dropout katmanı eklendi
model.add(Dense(units=1))
model.compile(optimizer='adam', loss='mean_squared_error')

# Modeli eğitme
model.fit(X_train, y_train, epochs=100, batch_size=16, validation_data=(X_test, y_test))

# Test seti üzerinde tahmin yapma
test_predictions = model.predict(X_test)
r2_test = r2_score(y_test, test_predictions)

print(f'Modelin Test R-kare skoru: {r2_test}')


Epoch 1/100
144/144 [==============================] - 16s 86ms/step - loss: 0.0134 - val_loss: 0.0018
Epoch 2/100
144/144 [==============================] - 13s 87ms/step - loss: 0.0042 - val_loss: 0.0017
Epoch 3/100
144/144 [==============================] - 12s 82ms/step - loss: 0.0033 - val_loss: 0.0015
Epoch 4/100
144/144 [==============================] - 12s 82ms/step - loss: 0.0032 - val_loss: 0.0012
Epoch 5/100
144/144 [==============================] - 12s 82ms/step - loss: 0.0029 - val_loss: 0.0017
Epoch 6/100
144/144 [==============================] - 12s 84ms/step - loss: 0.0026 - val_loss: 0.0011
Epoch 7/100
144/144 [==============================] - 12s 84ms/step - loss: 0.0025 - val_loss: 0.0011
Epoch 8/100
144/144 [==============================] - 12s 82ms/step - loss: 0.0023 - val_loss: 0.0013
Epoch 9/100
144/144 [==============================] - 12s 84ms/step - loss: 0.0023 - val_loss: 0.0015
Epoch 10/100
144/144 [==============================] - 12s 81ms/step - l

In [ ]:
#Eğitilmiş  modeli  kaydetme
save_model(model, 'model9420.h5')

<ipython-input-24-e7313819af16>:2: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  save_model(model, 'model9420.h5')


In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

# Verileri yükle
data = pd.read_excel("train.xlsx")
data = data.drop(["id", "temp", "w_status"], axis=1)

# 'date' sütununu datetime formatına çevirme
data['date_v'] = pd.to_datetime(data['date_v'])

# 'date' sütununu Unix zaman damgasına çevirme
data['unix_time'] = data['date_v'].apply(lambda x: int((x - datetime(1970, 1, 1)).total_seconds()))

# Gereksiz sütunu ve orijinal tarih sütununu kaldırma
data = data.drop(["date_v"], axis=1)

# MinMaxScaler ile 'yukh', 'reg' ve 'power' sütunlarını ölçeklendirme
scaler_yukh = MinMaxScaler()
scaler_reg = MinMaxScaler()
scaler_power = MinMaxScaler()

data['yukh_scaled'] = scaler_yukh.fit_transform(data[['yukh']])
data['reg_scaled'] = scaler_reg.fit_transform(data[['reg']])
data['power_scaled'] = scaler_power.fit_transform(data[['power']])

# Ölçeklenmiş sütunları veriden kaldırma
data = data.drop(["yukh", "reg", "power"], axis=1)

# Zaman serisi formatına dönüştürme
def create_time_series_data(data, window_size):
    X, y = [], []

    for i in range(len(data) - window_size):
        # Pencere boyutu kadar veriyi seçme
        window_yukh = data['yukh_scaled'][i:i+window_size].values
        window_reg = data['reg_scaled'][i:i+window_size].values
        window_power = data['power_scaled'][i:i+window_size].values
        target = data['yukh_scaled'][i + window_size]

        X.append(np.column_stack((window_yukh, window_reg, window_power)))
        y.append(target)

    return np.array(X), np.array(y)

# Pencere boyutunu belirleme
window_size = 64

# Giriş ve hedef verilerini oluşturma
X, y = create_time_series_data(data, window_size)

# Veriyi eğitim ve test setlerine ayırma
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

# Yüklü modeli yükleme
loaded_model = load_model('model.h5')

# Modeli tekrar eğitme
loaded_model.fit(X_train, y_train, epochs=50, batch_size=16, validation_data=(X_test, y_test))

# Test seti üzerinde tahmin yapma
test_predictions = loaded_model.predict(X_test)
r2_test = r2_score(y_test, test_predictions)

print(f'Modelin Test R-kare skoru: {r2_test}')


Epoch 1/50
144/144 [==============================] - 18s 106ms/step - loss: 4.9845e-04 - val_loss: 2.8164e-04
Epoch 2/50
144/144 [==============================] - 13s 93ms/step - loss: 4.4288e-04 - val_loss: 3.4596e-04
Epoch 3/50
144/144 [==============================] - 12s 87ms/step - loss: 5.1612e-04 - val_loss: 2.8380e-04
Epoch 4/50
144/144 [==============================] - 13s 90ms/step - loss: 5.4913e-04 - val_loss: 2.7967e-04
Epoch 5/50
144/144 [==============================] - 13s 94ms/step - loss: 4.4344e-04 - val_loss: 2.6001e-04
Epoch 6/50
144/144 [==============================] - 13s 93ms/step - loss: 4.9277e-04 - val_loss: 2.6006e-04
Epoch 7/50
144/144 [==============================] - 13s 88ms/step - loss: 4.4534e-04 - val_loss: 3.1887e-04
Epoch 8/50
144/144 [==============================] - 14s 100ms/step - loss: 5.0260e-04 - val_loss: 2.8661e-04
Epoch 9/50
144/144 [==============================] - 13s 93ms/step - loss: 4.3565e-04 - val_loss: 3.8358e-04
Epoch 10

In [ ]:
#Eğitilmiş  modeli  kaydetme
save_model(loaded_model, 'model9481.h5')

<ipython-input-23-800e174dfaed>:2: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  save_model(loaded_model, 'model9481.h5')


In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score
from keras.models import load_model

# Veriyi yükleme ve gereksiz sütunları kaldırma
data = pd.read_excel("test.xlsx")
data = data.drop(["id", "temp", "w_status"], axis=1)

data = data[(data != 0).all(axis=1)]  # Tüm sütunlar için 0 olmayan satırları seçme
data = data.dropna()  # NaN değerleri içeren satırları kaldırma

# 'date' sütununu datetime formatına çevirme
data['date_v'] = pd.to_datetime(data['date_v'])

# 'date' sütununu Unix zaman damgasına çevirme
data['unix_time'] = data['date_v'].apply(lambda x: int((x - datetime(1970, 1, 1)).total_seconds()))

# Gereksiz sütunu ve orijinal tarih sütununu kaldırma
data = data.drop(["date_v"], axis=1)

# MinMaxScaler ile 'yukh', 'reg' ve 'power' sütunlarını ölçeklendirme
scaler_yukh = MinMaxScaler()
scaler_reg = MinMaxScaler()
scaler_power = MinMaxScaler()

data['yukh_scaled'] = scaler_yukh.fit_transform(data[['yukh']])
data['reg_scaled'] = scaler_reg.fit_transform(data[['reg']])
data['power_scaled'] = scaler_power.fit_transform(data[['power']])

# Ölçeklenmiş sütunları veriden kaldırma
data = data.drop(["yukh", "reg", "power"], axis=1)

# Zaman serisi formatına dönüştürme
def create_time_series_data(data, window_size):
    X, y = [], []

    for i in range(len(data) - window_size):
        # Pencere boyutu kadar veriyi seçme
        window_yukh = data['yukh_scaled'][i:i+window_size].values
        window_reg = data['reg_scaled'][i:i+window_size].values
        window_power = data['power_scaled'][i:i+window_size].values
        target = data['yukh_scaled'][i: i + window_size]

        X.append(np.column_stack((window_yukh, window_reg, window_power)))
        y.append(target)

    return np.array(X), np.array(y)

# Pencere boyutunu belirleme (aynı boyutu kullanmalısınız)
window_size = 64

# Giriş ve hedef verilerini oluşturma
X, y = create_time_series_data(data, window_size)

# Modeli yükleme
model = load_model('model9481.h5')

# Test seti üzerinde tahmin yapma
test_predictions = model.predict(X)

# Test setindeki pencere boyutunda son gerçek değerler
y_true = y[:, -1]

# Test setindeki pencere boyutunda son tahminler
y_pred = test_predictions[:, -1]

# R-kare skoru hesaplama
r2_test = r2_score(y_true, y_pred)

print(f'Modelin Test R-kare skoru: {r2_test}')

26/26 [==============================] - 2s 36ms/step
Modelin Test R-kare skoru: 0.9739492825054626


In [ ]:
# Gerçek 'yukh' değerlerini al
yukh_original = scaler_yukh.inverse_transform(y[:, -1].reshape(-1, 1)).flatten()

# Tahmin edilen 'yukh' değerlerini ölçeklendirmeyi tersine çevirme
predicted_yukh_original = scaler_yukh.inverse_transform(test_predictions[:, -1].reshape(-1, 1)).flatten()

# Uzunlukları kontrol et
print(len(yukh_original), len(predicted_yukh_original))

# Veri çerçevesi oluşturma
result_df = pd.DataFrame({'Gerçek Yukh': yukh_original, 'Tahmin Yukh': predicted_yukh_original})

# Sonuçları ekrana yazdırma
print(result_df)

# Veriyi Excel dosyasına yazma
result_df.to_excel("gercek_tahmin_yukh1.xlsx", index=False)


812 812
     Gerçek Yukh  Tahmin Yukh
0         6047.0  6047.085449
1         6044.0  6045.154297
2         6045.0  6045.241699
3         6046.0  6045.894043
4         6043.0  6044.343262
..           ...          ...
807       6007.0  6007.411133
808       6006.0  6004.934082
809       6005.0  6003.037109
810       6004.0  6002.067383
811       6005.0  6002.730469

[812 rows x 2 columns]
